# Import Libraries

In [64]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import plotly.express as px
import plotly.io as pio
import plotly
from ydata_profiling import  ProfileReport
import PyQt5 as qt
from IPython.display import display, Markdown
#Enable graphing inside jupytor
#pip install PyQt5 #Install it if not installed
get_ipython().run_line_magic('matplotlib', 'inline')
matplotlib.get_backend()
from wordcloud import WordCloud
import arabic_reshaper
from bidi.algorithm import get_display

# Load Data

In [65]:
df = pd.read_excel('maintenance_cleaned_extended.xlsx')

# Visualization Functions

## Bar, Scatter, Line charts

In [66]:
def myPlot(data, plotType, title, x_label, y_label):
    data = data.sort_values(by=data.values,ascending=True)
    xs = data.index.astype(str)  # Convert index to strings for x-axis
    ys = data.values  # y-axis values
    # Calculate mean
    mean_value = ys.mean()

    # Get only values above the mean
    above_mean_data = data[data > mean_value]
    # Get only values below the mean
    below_mean_data = data[data < mean_value]

    # Sort values above mean in descending order and select top 3
    top3_above_mean = above_mean_data.sort_values(ascending=False).head(3)
    # Sort values below mean in descending order and select bottom 3
    bot3_below_mean = below_mean_data.sort_values(ascending=False).tail(3)

    # Create color mapping: blue for top 3 above mean, red for bottom 3 below mean, gray for others
    colors = ['blue' if index in top3_above_mean.index 
              else 'red' if index in bot3_below_mean.index 
              else 'gray' 
              for index in data.index]

    # Generate notes/annotations for each bar
    annotations = ['> Avg' if value > mean_value else '< Avg' for value in ys]

    if plotType == 'bar':
        fig = px.bar(x=xs, y=ys, title=title + ' Analysis')
        fig.update_traces(marker_color=colors)  # Bar-specific color update
        '''
        # Add annotations beside each bar
        for i, value in enumerate(ys):
            fig.add_annotation(
                x=xs[i], y=value, 
                text=annotations[i],  # Annotation text
                showarrow=False, 
                xanchor='center', 
                yanchor='bottom',
                font=dict(color=colors[i], size=12),  # Color annotations to match bar color
                #bgcolor="white",  # Optional background color for better visibility
                #bordercolor=colors[i]  # Match border color with text
            )
        '''    
    elif plotType == 'scatter':
        fig = px.scatter(x=xs, y=ys, title=title + ' Analysis')
        fig.update_traces(marker=dict(color=colors))  # Scatter-specific color update
        '''
        # Add annotations beside each scatter point
        for i, value in enumerate(ys):
            fig.add_annotation(
                x=xs[i], y=value, 
                text=annotations[i],
                showarrow=True, 
                arrowhead=2, 
                ax=20, ay=-20,
                font=dict(color=colors[i], size=12),
                #bgcolor="white",
                #bordercolor=colors[i]
            )
        '''    
    elif plotType == 'pie':
        fig = px.pie(names=xs, values=ys, title=title + ' Analysis')
        fig.update_traces(marker=dict(colors=colors))  # Pie-specific color update

        # Pie charts don't support annotations in the same way; can consider labels in the pie
    elif plotType == 'line':
        fig = px.line(x=xs, y=ys, title=title + ' Analysis')

    # Update layout for custom axis labels
    fig.update_layout(
        title_x=0.0,
        xaxis_title=x_label,  # Custom x-axis label
        yaxis_title=y_label   # Custom y-axis label
    )

    fig.show()

In [67]:
def myPlot1(data, xs, ys, clr, plotType, title, sort_by=None, ascending=True):
    if sort_by is not None:
        data_sorted = data.sort_values(by=sort_by, ascending=ascending)
    else:
        data_sorted = data
    xt = str(xs)
    yt = str(ys)
    clrt = str(clr)
    xs = data_sorted[xs]
    ys = data_sorted[ys]
    clr = data_sorted[clr].astype(str) if clr else None
    if plotType == 'bar':
        fig = px.bar(x=xs, y=ys, color=clr, title=title + ' Analysis')
    elif plotType == 'scatter':
        fig = px.scatter(x=xs, y=ys, color=clr, title=title + ' Analysis')
    elif plotType == 'line':
        fig = px.line(x=xs, y=ys, color=clr, title=title + ' Analysis')
    fig.update_layout(title_x=0.0)
    fig.update_layout(xaxis_title=xt, yaxis_title=yt)
    fig.update_layout(legend_title_text=clrt)
    fig.show()

In [68]:
def myPlot2(data, plotType, title):
    xs = data.index.astype(str)  # Index (x-axis)
    ys = data.values             # Values (y-axis)
    # Plot based on the plotType
    if plotType == 'bar':
        fig = px.bar(x=xs, y=ys, color=ys, title=title + ' Analysis')
    elif plotType == 'scatter':
        fig = px.scatter(x=xs, y=ys, color=ys, title=title + ' Analysis')
    elif plotType == 'line':
        fig = px.line(x=xs, y=ys, title=title + ' Analysis')
    # Center the title
    fig.update_layout(title_x=0.5)
    fig.show()

In [69]:
def myBoxPlot(data,x,y,color,title):
    fig = px.box(data, x=x, y=y, color=color, title=title)
    fig.update_layout(
        title_x=0.5,
        xaxis_title=str(x),
        yaxis_title=str(y)
    )
    fig.show()

## Sunburst chart

In [70]:
def mySunBurst(data, name1,name2, value, title):
    fig = px.sunburst(
        data_frame=data,
        path=[name1,name2],   # Add both cost_category and damage type to the hierarchy
        #path=name,
        values=value,  # Define the values (damage_count)
        title=title +' Analysis'
    )
    fig.update_layout(title_x=0.0)
    fig.show()

## Pie chart

In [71]:
def myPie(data,title_prefix):
    name  = data.index
    value = data.values
    fig = px.pie(data_frame=data,
                 names = name, 
                 values = value,
                 title ='Top 5 '+ title_prefix +' Analysis'
                )
    fig.update_layout(title_x=0.5)

    fig.show()

## Combine DataFrames

In [72]:
def combine(data,first_field,first_field_count,field_grouped_on,resulting_field_value):
    data_first_cat = data[first_field].value_counts().reset_index()
    data_first_cat.columns = [first_field,first_field_count]
    data_merged = data.groupby([first_field])[field_grouped_on].sum().reset_index(name=resulting_field_value)
    data_merged = data_merged.merge(data_first_cat,on=first_field)
    return first_field_count, resulting_field_value, data_merged

## Bi - Variance Analysis

### Service Duration Efficiency

#### Service duration for each service type

In [73]:
text='Observations:'
display(Markdown('###### '+text ))
text ='1. Concentration of short durations: The majority of services are completed in less than 15 days, with a large number of them completed within 0-5 days. This indicates that overall service efficiency is high in most cases.'
display(Markdown(text))
text ='2. Recurring delays: Some services extend to 20, 30, and even 70 days, suggesting potential bottlenecks or inefficiencies affecting a minority of services'
display(Markdown(text))

Service_Duration = df.groupby(['service_duration'])['service_duration'].sum().reset_index(name='Total Service Duration')
Service_Duration = Service_Duration.sort_values('Total Service Duration',ascending=True)
# Create the updated bar chart
myPlot1(Service_Duration, 'service_duration', 'Total Service Duration', None, 'bar', 'Service duration for each service type', sort_by=None, ascending=True)

text = 'Recommendations:'
display(Markdown('###### '+text ))
text = '1. Investigate Parts and Material Availability:'
display(Markdown(text))
text = '* Action: Examine whether these longer service durations (20+ days) are linked to delays in receiving spare parts. It’s possible that locations experiencing these delays have supply chain issues.'
display(Markdown(text))
text = '* Track: Track the time taken to order and receive parts for each service type.'
display(Markdown(text))
text = '2. Damage Complexity:'
display(Markdown(text))
text = '* Action: Longer service durations may also be related to the complexity of the damage. Categorize service types by complexity to understand if certain repair types inherently take longer.'
display(Markdown(text))
text = '3. Inventory Management:'
display(Markdown(text))
text = '* Action: Analyze whether these longer service types are linked to low inventory levels of critical parts at specific locations. Improving stock management might reduce these longer durations.'
display(Markdown(text))
text = '4. Worker and Resource Allocation:'
display(Markdown(text))
text = '* Action: Consider whether locations that regularly experience long service times are understaffed or if expertise mismatches are slowing down complex repairs.'
display(Markdown(text))



###### Observations:

1. Concentration of short durations: The majority of services are completed in less than 15 days, with a large number of them completed within 0-5 days. This indicates that overall service efficiency is high in most cases.

2. Recurring delays: Some services extend to 20, 30, and even 70 days, suggesting potential bottlenecks or inefficiencies affecting a minority of services

###### Recommendations:

1. Investigate Parts and Material Availability:

...

* Action: Longer service durations may also be related to the complexity of the damage. Categorize service types by complexity to understand if certain repair types inherently take longer.

3. Inventory Management:

* Action: Analyze whether these longer service types are linked to low inventory levels of critical parts at specific locations. Improving stock management might reduce these longer durations.

4. Worker and Resource Allocation:

* Action: Consider whether locations that regularly experience long service times are understaffed or if expertise mismatches are slowing down complex repairs.

#### Service duration per damage type

In [74]:
text='Observations:'
display(Markdown('###### ' +text))
text='* Short durations across damage types: Similar to the first chart, many services across various damage types are completed in less than 15 days.'
display(Markdown(text))
text='* Certain damage types with delays: Some damage types, particularly represented by colors like orange and red, extend to longer durations (20+ days), suggesting that certain types of repairs might be taking disproportionately long.'
display(Markdown(text))

Service_Duration = df.groupby(['service_duration','damage type'])['service_duration'].sum().reset_index(name='Total Service Duration')
Service_Duration = Service_Duration.sort_values('Total Service Duration',ascending=True)
# Create the updated bar chart
myPlot1(Service_Duration, 'service_duration', 'Total Service Duration', 'damage type', 'bar', 'Service duration for each service type per damage type', sort_by=None, ascending=True)

text = 'Recommendations:'
display(Markdown('###### ' +text))
text='1. Investigate Parts and Material Availability:'
display(Markdown(text))
text='* Action: Determine if certain damage types (such as those associated with orange/red categories) consistently face delays due to parts availability.'
display(Markdown(text))
text='* Track: Monitoring the supply chain of specific parts linked to these damage types may help reduce service delays.'
display(Markdown(text))
text='2. Damage Complexity:'
display(Markdown(text))
text='* Action: Compare the complexity of different damage types and investigate whether the longer durations for specific damage types (like orange/red) are due to inherent complexity. For instance, complex mechanical or electrical issues could naturally require more time.'
display(Markdown(text))
text='3.	Client-Specific Preferences:'
display(Markdown(text))
text='* Action: Check whether certain damage types are tied to specific clients or special handling requests, which could be contributing to extended service durations. Special client requirements might be extending repair times.'
display(Markdown(text))
text='4. Worker and Resource Allocation:'
display(Markdown(text))
text='* Action: Evaluate whether locations that handle certain complex repairs (orange/red damage types) have the necessary skilled workforce and tools to complete the repairs more efficiently.'
display(Markdown(text))

###### Observations:

* Short durations across damage types: Similar to the first chart, many services across various damage types are completed in less than 15 days.

* Certain damage types with delays: Some damage types, particularly represented by colors like orange and red, extend to longer durations (20+ days), suggesting that certain types of repairs might be taking disproportionately long.

###### Recommendations:

1. Investigate Parts and Material Availability:

...

* Action: Compare the complexity of different damage types and investigate whether the longer durations for specific damage types (like orange/red) are due to inherent complexity. For instance, complex mechanical or electrical issues could naturally require more time.

3.	Client-Specific Preferences:

* Action: Check whether certain damage types are tied to specific clients or special handling requests, which could be contributing to extended service durations. Special client requirements might be extending repair times.

4. Worker and Resource Allocation:

* Action: Evaluate whether locations that handle certain complex repairs (orange/red damage types) have the necessary skilled workforce and tools to complete the repairs more efficiently.

#### Service duration per damage type per location

In [75]:
text = 'Observations:'
display(Markdown('###### ' +text))
text='* High concentration at specific locations: Certain locations (indicated by large bubbles, particularly in orange) handle a larger volume of repairs, some of which take significantly longer (20+ days). Other locations have fewer services or shorter durations.'
display(Markdown(text))
text='* Location bottlenecks: Some locations are outliers, with high service counts but extended durations, while others seem underutilized.'
display(Markdown(text))

Service_Duration = df.groupby(['service_duration','damage type','location'])['service_duration'].sum().reset_index(name='Total Service Duration')
Service_Duration = Service_Duration.sort_values('Total Service Duration',ascending=True)
# Create the updated bar chart

fig = px.scatter(Service_Duration, 
             x='location', 
             y='Total Service Duration',
             color='damage type',
             size='service_duration',
             title="Service duration for each service type for each location",
             labels={'service_duration': 'Service Duration Days', 'Total Service Duration': 'Total Service Duration','damage type': 'Damage Type'})

fig.update_layout(xaxis_title="Service Location Days", yaxis_title="Service Duration Count", 
                  xaxis_tickangle=-45, height=500)
fig.show()
text='Recommendations:'
display(Markdown('###### ' +text))
text='1. Investigate Parts and Material Availability:'
display(Markdown(text))
text='* Action: Locations with larger bubbles and longer service durations might be experiencing delays due to parts availability. Investigating their supply chains and inventory systems can help identify potential delays.'
display(Markdown(text))
text='2. Inventory Management:'
display(Markdown(text))
text='* Action: Locations with higher service volumes (larger bubbles) may require more efficient inventory management to ensure parts are always available when needed.'
display(Markdown(text))
text='* Optimize: Ensure that high-demand locations are prioritized for faster restocking and have a well-managed parts inventory to prevent service delays.'
display(Markdown(text))
text='3. Worker and Resource Allocation:'
display(Markdown(text))
text='* tion: Assess whether the locations with larger bubbles and longer durations are sufficiently staffed and whether the expertise of the workers matches the types of services required. If these locations are understaffed or ill-equipped, allocating more resources could reduce delays.'
display(Markdown(text))
text='* Adjustment: Redistribute the workload between busier locations (with long durations) and underutilized ones to balance resource use and reduce service bottlenecks.'
display(Markdown(text))
text='4. Client-Specific Preferences:'
display(Markdown(text))
text='* Action: Some locations might serve specific high-value clients who have custom requirements, leading to longer service durations. Review service agreements to identify if these requirements are necessary or if they could be streamlined to reduce delays.'
display(Markdown(text))


###### Observations:

* High concentration at specific locations: Certain locations (indicated by large bubbles, particularly in orange) handle a larger volume of repairs, some of which take significantly longer (20+ days). Other locations have fewer services or shorter durations.

* Location bottlenecks: Some locations are outliers, with high service counts but extended durations, while others seem underutilized.

###### Recommendations:

1. Investigate Parts and Material Availability:

...

3. Worker and Resource Allocation:

* tion: Assess whether the locations with larger bubbles and longer durations are sufficiently staffed and whether the expertise of the workers matches the types of services required. If these locations are understaffed or ill-equipped, allocating more resources could reduce delays.

* Adjustment: Redistribute the workload between busier locations (with long durations) and underutilized ones to balance resource use and reduce service bottlenecks.

4. Client-Specific Preferences:

* Action: Some locations might serve specific high-value clients who have custom requirements, leading to longer service durations. Review service agreements to identify if these requirements are necessary or if they could be streamlined to reduce delays.

###### Final Recommendations (Across All Charts):
1.	Parts and Material Availability: Delays in receiving parts may be a significant factor contributing to longer service durations. Tracking how long it takes to receive parts for each service type and location can reveal where supply chain improvements are needed.

2.	Damage Complexity: Longer service durations could be due to the inherent complexity of certain repairs. By categorizing and analyzing each damage type’s complexity, Ahmad can ensure that longer durations are not mistaken for inefficiency and adjust expectations accordingly.

3.	Inventory Management: Ensuring that commonly needed parts are readily available at all locations can help prevent delays. Implement an inventory management system that automatically restocks critical parts to minimize wait times.

4.	Client-Specific Preferences: Review service contracts and client requirements to determine if special requests are unnecessarily lengthening service durations. Streamlining these requirements could speed up processes without compromising client satisfaction.

5.	Worker and Resource Allocation: Proper allocation of workers and tools, especially in high-traffic locations, can ensure that the right resources are applied to the right jobs. Consider upskilling workers or increasing staff levels where necessary to reduce delays.

###### By addressing these factors holistically—supply chain efficiency, damage complexity, inventory management, client demands, and worker allocation—Ahmad can significantly improve service throughput, reduce delays, and increase overall income.
